In [1]:


import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 

import numpy as np
import tensorflow as tf
import pandas as pd
import pyarabic.araby as araby
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
import torch
from sklearn.metrics import accuracy_score, f1_score
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, BertForSequenceClassification
from datasets import load_dataset, Dataset, concatenate_datasets
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 1000)


log_file = 'HARD_balanced.txt'
with open(log_file, 'w') as f:
    f.write('Model,Accuracy,F1\n')



# ds = load_dataset('hard')

df = pd.read_csv('HARD/balanced-reviews.txt', encoding='utf-16', engine='python', sep='\t') #, quotechar="'"
# df = pd.read_csv('HARD/unbalanced-reviews.txt', encoding='utf-16', engine='python', sep='\t') #, quotechar="'"
# display(df.columns)
df = df[['rating', 'review']]

df['rating'].replace({1: 0, 2: 0}, inplace=True) #replace 1 and 2 with 0 (negative)
# df['rating'].replace({3: 1}, inplace=True)  #replace 3 with 1 (neutral)
df['rating'].replace({4: 2, 5: 1}, inplace=True)  #replace 4 and 5 with 1 (positive)
df['label'] = df['rating']

classes = set(df['rating'].values)
display(classes)
classes_num = len(classes)
display(classes_num)
display(len(df))

display(df[:4])


ds = Dataset.from_pandas(df)

# ds = ds['train']
ds = ds.train_test_split(test_size=0.2)
display(ds)
df = ''

max_sequence_length = 128

# classes_num = 6
# display(classes_num)
# display(ds)


models = ['faisalq/bert-base-arabic-wordpiece', 'faisalq/bert-base-arabic-senpiece',
          'faisalq/bert-base-arabic-bbpe']


for model_name in models:
    for i in range(3):
        print(f'{model_name}, try:{i}')
              
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = BertForSequenceClassification.from_pretrained(model_name,
                                                              num_labels=classes_num).to('cuda')                                                 
        dataset_train = ds['train']
        dataset_validation = ds['test']                                                     
        
      

        def preprocess_function(examples):
            return tokenizer(examples['review'], truncation=True, padding="max_length",
                            max_length=max_sequence_length)
        
        
        dataset_train = dataset_train.map(preprocess_function, batched=True)
        dataset_validation = dataset_validation.map(preprocess_function, batched=True)
        
       
        
        def compute_metrics(eval_pred):
            logits, labels = eval_pred
            predictions = np.argmax(logits, axis=-1)    
            acc = accuracy_score(labels, predictions)        
            f1 = f1_score(labels, predictions, average='macro')   
            with open(log_file, 'a') as f:
                f.write(f'{model_name},{acc},{f1}\n')
            return {'accuracy': acc, 'f1_score': f1}


        
        
        epochs = 5
        save_steps = 10000 #save checkpoint every 10000 steps
        batch_size = 256
        
        training_args = TrainingArguments(
            output_dir = 'bert/',
            overwrite_output_dir=True,
            num_train_epochs = epochs,
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            save_steps = save_steps,
            save_total_limit = 1, #only save the last 5 checkpoints
            fp16=True,
            learning_rate = 5e-5,  # 5e-5 is the default
            logging_steps = 100, #50_000
            evaluation_strategy = 'steps',
            # evaluate_during_training = True,
            eval_steps = 100
            
        )
        
        trainer = Trainer(
            model = model,
            args = training_args,
            # data_collator=data_collator,
            train_dataset=dataset_train,
            eval_dataset=dataset_validation,
            compute_metrics = compute_metrics
        )
        
        
        # trainer.train(resume_from_checkpoint=True)
        trainer.train()


results = pd.read_csv(log_file)

best_results = results.groupby('Model', as_index=False)['F1'].max()

best_results = pd.merge(best_results, results, on=['Model', 'F1'])
best_results = best_results[['Model', 'Accuracy', 'F1']]
best_results = best_results.drop_duplicates()
best_results.to_csv('HARD_balanced_results.csv')
display(best_results)



2024-02-18 04:51:24.233082: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-18 04:51:24.256682: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-18 04:51:24.605309: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


{0, 1, 2}

3

105698

,rating,review,label
0,0,“ممتاز”. النظافة والطاقم متعاون.,0
1,1,استثنائي. سهولة إنهاء المعاملة في الاستقبال. لاشيئ,1
2,1,استثنائي. انصح بأختيار الاسويت و بالاخص غرفه رقم 801. نوعية الارضيه,1
3,0,“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق 2 نجمه,0


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at faisalq/bert-base-arabic-bbpe and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/84558 [00:00<?, ? examples/s]

Map:   0%|          | 0/21140 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss,Accuracy,F1 Score
100,0.362700,0.276799,0.880085,0.856517
200,0.276400,0.266082,0.887228,0.864310
300,0.272200,0.273791,0.887181,0.862528
400,0.241900,0.262058,0.890445,0.868463
500,0.233900,0.262488,0.890445,0.867431
600,0.234500,0.264067,0.884957,0.862111
700,0.217500,0.275083,0.889593,0.866657
800,0.194900,0.284300,0.885714,0.863679
900,0.180400,0.280625,0.888269,0.864708
1000,0.189000,0.294090,0.888174,0.863928


,Model,Accuracy,F1
0,faisalq/bert-base-arabic-bbpe,0.890445,0.868463
3,faisalq/bert-base-arabic-senpiece,0.891060,0.868155
6,faisalq/bert-base-arabic-wordpiece,0.890445,0.868443
